In [35]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
LabEncoder = LabelEncoder()

In [60]:
sales_data = pd.read_csv('Train_UWu5bXk.csv')

In [61]:
meanbytype=sales_data.groupby('Item_Type').Item_Weight.transform('mean')
sales_data['Item_Weight'].fillna(meanbytype,inplace=True)
sales_data['Outlet_Size'].fillna('Missing',inplace=True)

In [62]:
# get the age of Outlet
import datetime 
datetime.datetime.now().year
sales_data['Outlet_Age']=sales_data.Outlet_Establishment_Year.sub(datetime.datetime.now().year).abs()
#Encode outlet size ordinally
scale_size={'Missing':1,'Small':1,'Medium':2,'High':3}
sales_data['Outlet_Size']=sales_data['Outlet_Size'].replace(scale_size)
#clean fat content column
sales_data.Item_Fat_Content=sales_data.Item_Fat_Content.str.upper()
scale_fat_content={'LOW FAT':0,'LF':0,'REGULAR':1,'REG':1}
sales_data['Item_Fat_Content']=sales_data['Item_Fat_Content'].replace(scale_fat_content)
sales_data.Outlet_Location_Type=pd.to_numeric(sales_data.Outlet_Location_Type.str.replace('Tier ',''))
scale_fat_outlet_type={'Grocery Store':0,'Supermarket Type1':1,'Supermarket Type2':2,'Supermarket Type3':3}
sales_data['Outlet_Type']=sales_data['Outlet_Type'].replace(scale_fat_outlet_type)
# sales_data.Item_Type=LabEncoder.fit_transform(sales_data.Item_Type)
sales_data=pd.concat([sales_data,pd.get_dummies(sales_data.Item_Type,prefix='Item_Type')],axis=1)
# sales_data=pd.concat([sales_data,pd.get_dummies(sales_data.Item_Type,prefix='Item_Identifier')],axis=1)
sales_data['Item_Ident_prefix']=sales_data.Item_Identifier.str.slice(start=0,stop=2)
sales_data=pd.concat([sales_data,pd.get_dummies(sales_data.Item_Ident_prefix,prefix='Item_Identifier')],axis=1)
# sales_data['Item_Ident_post']=sales_data.Item_Identifier.str.slice(start=2,stop=3)
sales_data=pd.concat([sales_data,pd.get_dummies(sales_data.Outlet_Identifier,prefix='Outlet_Identifier')],axis=1)
sales_data=sales_data.drop(['Item_Identifier','Outlet_Establishment_Year','Item_Type','Outlet_Identifier','Item_Ident_prefix'],axis=1)

In [63]:
y=sales_data['Item_Outlet_Sales'].values
X=sales_data.drop('Item_Outlet_Sales',axis=1).values

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_std=ss.fit_transform(X)

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X_std,y,test_size=0.25,random_state=0)

In [64]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(xtrain,ytrain)
pred_train=linreg.predict(xtrain)
pred_test=linreg.predict(xtest)
from sklearn.metrics import mean_squared_error
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print("Train Error",lin_rmse_train,"Test Error",lin_rmse_test)

Train Error 1121.2164292578823 Test Error 1147.723288248753


In [65]:
from sklearn.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(3)
X_Poly=pf.fit_transform(X_std)

In [66]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X_Poly,y,test_size=0.25,random_state=0)
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(xtrain,ytrain)
pred_train=linreg.predict(xtrain)
pred_test=linreg.predict(xtest)
from sklearn.metrics import mean_squared_error
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print("Train Error",lin_rmse_train,"Test Error",lin_rmse_test)

Train Error 992.1308112646474 Test Error 131009410555358.97


In [78]:
from sklearn.linear_model import Ridge
alphas=[19950,19975,19999]#,20300,20400]
df=pd.DataFrame(columns=['Alpha','train_error','test_error'])
for alpha_ in alphas:
    rr=Ridge(alpha=alpha_)
    rr.fit(xtrain,ytrain)
    
    pred_train=rr.predict(xtrain)
    pred_test=rr.predict(xtest)
    lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
    lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
#     tup=[]
#     tup=[alpha_,lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train]
    print(alpha_,lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train)
#     df.append(pd.DataFrame(tup))

19950 1027.4723944462469 1108.9681738592005 81.49577941295365
19975 1027.5016424686196 1108.9683429582972 81.46670048967758
19999 1027.529701481023 1108.96853084639 81.43882936536693


In [89]:
from sklearn.linear_model import Ridge
ridg=Ridge(alpha=19950)
ridg.fit(xtrain,ytrain)
pred_train=rr.predict(xtrain)
pred_test=rr.predict(xtest)
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print(lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train)

1060.717047056385 1087.6663367940478 26.949289737662866


In [84]:
from sklearn.linear_model import Lasso
alphas=[55,60,70,75]
df=pd.DataFrame(columns=['Alpha','train_error','test_error'])
for alpha_ in alphas:
    rr=Lasso(alpha=alpha_)
    rr.fit(xtrain,ytrain)
    
    pred_train=rr.predict(xtrain)
    pred_test=rr.predict(xtest)
    lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
    lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
#     tup=[]
#     tup=[alpha_,lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train]  
    print(alpha_,lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train)

55 1056.0618365913765 1087.2977423929847 31.23590580160817
60 1057.217082270026 1087.2343464675605 30.017264197534587
70 1059.5707672218457 1087.4210161689964 27.850248947150703
75 1060.717047056385 1087.6663367940478 26.949289737662866


In [129]:
from sklearn.linear_model import ElasticNet
#alphas=[5.5,5.75,6,6.25,6.50,6.75,7]
l1_ratios=[0,0.25,0.5,0.75,1]
df=pd.DataFrame(columns=['Alpha','train_error','test_error'])
#for alpha_ in alphas:
    #rr=ElasticNet(alpha=alpha_)
for l1_ratio_ in l1_ratios:
    rr=ElasticNet(alpha=6,l1_ratio=l1_ratio_)
    rr.fit(xtrain,ytrain)
    
    pred_train=rr.predict(xtrain)
    pred_test=rr.predict(xtest)
    lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
    lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
#     tup=[]
#     tup=[alpha_,lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train]
#     df=df.append({'Alpha':alpha_,'train_error':lin_rmse_train,'test_error':lin_rmse_test},ignore_index=True)
#     print(alpha_,lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train)
    print(l1_ratio_,lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train)
    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0 1045.5838325675827 1113.5057794149636 67.92194684738092
0 1039.2572244800497 1108.8718191922974 69.61459471224771
0 1032.1284433128922 1105.4215311823014 73.29308786940919
0 1022.7751891706926 1104.411759016405 81.63656984571253


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0 1002.8599016492403 1122.7512736894093 119.89137204016902


In [ ]:
from sklearn.linear_model import ElasticNet
ridg=ElasticNet(alpha=19950)
ridg.fit(xtrain,ytrain)
pred_train=rr.predict(xtrain)
pred_test=rr.predict(xtest)
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print(lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train)
ridge.score()

In [122]:
sales_data_test=pd.read_csv('Test_u94Q5KV.csv')
sales_data_tmp=sales_data_test
meanbytype=sales_data_test.groupby('Item_Type').Item_Weight.transform('mean')
sales_data_test['Item_Weight'].fillna(meanbytype,inplace=True)
sales_data_test['Outlet_Size'].fillna('Missing',inplace=True)
# get the age of Outlet
import datetime 
datetime.datetime.now().year
sales_data_test['Outlet_Age']=sales_data_test.Outlet_Establishment_Year.sub(datetime.datetime.now().year).abs()
#Encode outlet size ordinally
scale_size={'Missing':1,'Small':1,'Medium':2,'High':3}
sales_data_test['Outlet_Size']=sales_data_test['Outlet_Size'].replace(scale_size)
#clean fat content column
sales_data_test.Item_Fat_Content=sales_data_test.Item_Fat_Content.str.upper()
scale_fat_content={'LOW FAT':0,'LF':0,'REGULAR':1,'REG':1}
sales_data_test['Item_Fat_Content']=sales_data_test['Item_Fat_Content'].replace(scale_fat_content)
sales_data_test.Outlet_Location_Type=pd.to_numeric(sales_data_test.Outlet_Location_Type.str.replace('Tier ',''))
scale_fat_outlet_type={'Grocery Store':0,'Supermarket Type1':1,'Supermarket Type2':2,'Supermarket Type3':3}
sales_data_test['Outlet_Type']=sales_data_test['Outlet_Type'].replace(scale_fat_outlet_type)
# sales_data_test.Item_Type=LabEncoder.fit_transform(sales_data_test.Item_Type)
sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Item_Type,prefix='Item_Type')],axis=1)
# sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Item_Type,prefix='Item_Identifier')],axis=1)
sales_data_test['Item_Ident_prefix']=sales_data_test.Item_Identifier.str.slice(start=0,stop=2)
sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Item_Ident_prefix,prefix='Item_Identifier')],axis=1)
# sales_data_test['Item_Ident_post']=sales_data_test.Item_Identifier.str.slice(start=2,stop=3)
sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Outlet_Identifier,prefix='Outlet_Identifier')],axis=1)
sales_data_test=sales_data_test.drop(['Item_Identifier','Outlet_Establishment_Year','Item_Type','Outlet_Identifier','Item_Ident_prefix'],axis=1)
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_std=ss.fit_transform(sales_data_test)
from sklearn.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(3)
X_Poly_test=pf.fit_transform(X_std)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [110]:
from sklearn.linear_model import Lasso
ridg=Lasso(alpha=60)
ridg.fit(xtrain,ytrain)
pred_train=rr.predict(xtrain)
pred_test=rr.predict(xtest)
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print(lin_rmse_train,lin_rmse_test,lin_rmse_test-lin_rmse_train)

1036.4169749579305 1105.6389358375316 69.22196087960106


In [123]:
sales_data_tmp['Item_Outlet_Sales']=ridg.predict(X_Poly_test)

In [127]:
final_Output=sales_data_tmp[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']]
final_Output.to_csv('BigDataMart_Predicted.csv')